### Junta os CSV

Junta todos os csv de um determinado diretório em um único csv

In [6]:
import pandas as pd
import os

In [11]:
def join_csv(pathFrom, fileTo, field='Fechamento'):
    counter = 0
    newCsv = pd.DataFrame()

    for file in os.listdir(pathFrom):
        counter = counter+1
        csv = pd.read_csv(pathFrom+'/'+file)
        
        # first column is Data
        if(counter == 1):
            newCsv['Data'] = csv['Data']

        name = os.path.splitext(file)[0]
        newCsv[name] = csv[field]

    newCsv.to_csv(fileTo)
    print('New file CSV successfully created')
    
def find_csv_max_lines(path):
    rows = []
    for file in os.listdir(path):
        with open(path+'/'+file) as f:
            lines = sum(1 for line in f)
            rows.append([file, lines])

    csv = pd.DataFrame(rows, columns=['File', 'Lines'])
    maxlines = csv['Lines'].max()
    return csv.loc[(csv['Lines'] == maxlines)]

def join_csv_merge(dfcsv, fileTo, path_old, path_older):
    newCsv = pd.DataFrame()
    define_data = False

    for i, row_merge in merge.iterrows():
        fe1 = pd.read_csv(path_older+'/'+row_merge.File)
        fe2 = pd.read_csv(path_old+'/'+row_merge.File)    
        dates=[]

        if (define_data == False):
            for i, row in fe1.iterrows():
                dates.append(row['Data'])
            for i, row in fe2.iterrows():
                dates.append(row['Data'])
            newCsv['Data'] = dates
            define_data = True

        rows=[]
        for i, row in fe1.iterrows():
            rows.append(row['Fechamento'])
        for i, row in fe2.iterrows():
            rows.append(row['Fechamento'])

        name = os.path.splitext(row_merge.File)[0]
        newCsv[name] = rows
        
    newCsv.to_csv(fileTo)

    print('New file CSV successfully created')

### Junta o preço de Fechamento de todos os csv em um único csv

In [13]:
# usando a função para juntar os csv's de 'datasets-b3'
join_csv('datasets-b3', 'datasets/data.csv')

New file CSV successfully created


### Junta o preço de Fechamento de todos os csv em um único csv de 2 diretório
Somente os csv que existem nos dois diretórios

In [8]:
csv19 = find_csv_max_lines('datasets-b3-2019')
csv20 = find_csv_max_lines('datasets-b3')
csv20.shape, csv19.shape

((266, 2), (204, 2))

In [9]:
merge = pd.merge(csv20, csv19, left_on='File', right_on='File')
merge.shape

(201, 3)

In [12]:
join_csv_merge(merge, 'datasets/data1.csv', 'datasets-b3', 'datasets-b3-2019')

New file CSV successfully created
